In [1]:
import json
import requests

In [6]:
def get_ai_score(text):
	url = 'https://contentatscale.ai/ai-content-detector/'
	headers = {
		'authority': 'contentatscale.ai',
		'accept': '*/*',
		'accept-language': 'en-US,en;q=0.9',
		'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
		'cookie': '_cas_stats=eyJVc2VyQWdlbnQiOiJNb3ppbGxhLzUuMCAoTWFjaW50b3NoOyBJbnRlbCBNYWMgT1MgWCAxMF8xNV83KSBBcHBsZVdlYktpdC81MzcuMzYgKEtIVE1MLCBsaWtlIEdlY2tvKSBDaHJvbWUvMTIyLjAuMC4wIFNhZmFyaS81MzcuMzYiLCJMYW5ndWFnZSI6ImVuLVVTIiwiU2NyZWVuV2lkdGgiOjE2OTIsIlNjcmVlbkhlaWdodCI6MzAwOCwiQ29sb3JEZXB0aCI6MjQsIlRpbWVab25lT2Zmc2V0IjoyNDAsIlBsdWdpbnMiOiJQREYgVmlld2VyLCBDaHJvbWUgUERGIFZpZXdlciwgQ2hyb21pdW0gUERGIFZpZXdlciwgTWljcm9zb2Z0IEVkZ2UgUERGIFZpZXdlciwgV2ViS2l0IGJ1aWx0LWluIFBERiJ9',
		'dnt': '1',
		'origin': 'https://contentatscale.ai',
		'referer': 'https://contentatscale.ai/ai-content-detector/',
		'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
		'sec-ch-ua-mobile': '?0',
		'sec-ch-ua-platform': '"macOS"',
		'sec-fetch-dest': 'empty',
		'sec-fetch-mode': 'cors',
		'sec-fetch-site': 'same-origin',
		'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
		'x-requested-with': 'XMLHttpRequest'
	}

	data = {
		'content': f"<p>{text}</p>",
		'action': 'checkaiscore'
	}

	response = requests.post(url, headers=headers, data=data)

	if response.status_code == 200:
		# Get JSON data from the response
		return response.json()
	else:
		raise Exception(f"Request was not successful. Status code: {response.status_code}")

In [8]:
get_ai_score("text")

{'status': 'Failure', 'message': 'Valid Text is required.'}